### Imports



In [8]:
import json
from transformers import BertTokenizer
import pandas as pd
import json
import os
# os.chdir(r'C:\Users\deepa\Downloads')

with open('./data/annotations.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

### load the dataset


In [4]:
with open('./data/classNERData.json', 'r', encoding='utf-8') as file:
    data = json.load(file)


In [9]:
##create BIO labels - for each sentence
def create_bio_labels(data):
    bio_labels = []
    for sentence in data:
        bio = 'O' * len(sentence['request'])
        for entity in sentence['entities']:
            start = entity['start']
            end = entity['end']
            label_type = entity['category']
            tmp = sentence['request'][start:end]
            for i, word in enumerate(tmp.split()):
                if i == 0:
                    bio = bio[:start] + 'B-' + label_type + bio[start + 2:]
                else:
                    bio = bio[:start] + 'I-' + label_type + bio[start + 2:]
                start += len(word) + 1
        bio_labels.append(bio)
    return bio_labels

data = [
    {
      "request": "I want to fly to New York on the 13.3",
      "entities": [
        {"start": 16, "end": 23, "text": "New York", "category": "DESTINATION"},
        {"start": 32, "end": 35, "text": "13.3", "category": "DATE"}
      ]
    } 
]

print(data['request'])
# bio_labels = create_bio_labels(data)
# print(bio_labels)

['OOOOOOOOOOOOOOOOOOB-I-ORII-ORIGB-DI-DESI-DESTB-DATEI-DATEIONNATIONTINATIONNIGINOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO']


In [5]:
#print all requests
for sentence in data:
    print(sentence['request'])

רציתי להזמין כרטיס טיסה עבור 3 נוסעים.
אני רוצה לטוס לסינגפור
אני רוצה להזמין כרטיס לרומא
האם ניתן להזמין כרטיס טיסה דרך הצ'אטבוט?
ברצוני להזמין כרטיס טיסה מתל אביב לניו יורק לתאריך 2024-05-01.
אני מעוניין לרכוש כרטיס טיסה שתצא מברצלונה ותגיע ליוון ב10.2
בתאריך ה10 למרץ אני רוצה לטוס ליוטה
באיזה מחיר ניתן להזמין כרטיס טיסה הלוך ושוב לפריז?
רציתי להזמין כרטיס טיסה לילד קטן.
ברצוני להזמין טיסה לספרד ליום חמישי
כמה עולה כרטיס טיסה לברצלונה?
אני רוצה להזמין טיסה להיום בלילה
בא לי 7 מושבים למדריד
אשמח להזמין טיסה שיוצאת עד שעה 18
יש טיסות הלוך לרומניה שיוצאת מחר בבוקר?
אני צריך שני מושבים ללונדון
אני רוצה לעשות הזמנה של טיסה
איך מזמינים טיסה למחר בבוקר?
מה המחיר של טיסה לברלין?
אני אשמח להזמין כרטיס טיסה לנכד שלי


### Tokenize the sentences


In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenized_texts = []
labels = []


for example in data:
    tokenized_text = tokenizer.tokenize(example['request'])
    tokenized_texts.append(tokenized_text)
    labels.append(example['entities'])

In [67]:
# data = [
#     {
#       "request": "I want to fly to New York on the 13.3",
#       "entities": [
#         {"start": 17, "end": 25, "text": "New York", "category": "DESTINATION"},
#         {"start": 33, "end": 37, "text": "13.3", "category": "DATE"}
#       ]
#     } 
# ]

data = [
    {
      "request": "ברצוני להזמין כרטיס טיסה מתל אביב לניו יורק לתאריך 2024-05-01.",
      "class": 0,
      "entities": [
        {"start": 25, "end": 33, "text": "מתל אביב", "category": "ORIGIN"},
        {"start": 35, "end": 43, "text": "ניו יורק", "category": "DESTINATION"},
        {"start": 51, "end": 60, "text": "2024-05-01", "category": "DATE"}
      ]
    }
]
print(len(data[0]['request']))
print(data[0]['request'][35:43])


62
ניו יורק


In [49]:

data = [
    {
      "request": "I want to fly to New York on the 13.3",
      "entities": [
        {"start": 17, "end": 25, "text": "New York", "category": "DESTINATION"},
        {"start": 33, "end": 37, "text": "13.3", "category": "DATE"}
      ]
    } 
]

# Begeining Inside Outside
# BIO label - O O O O O B-DESTINATION I-DESTINATION O O B-DATE

# a function that replace each word in a sentence with its corresponding indexes in the sentecnce
def replace_words_with_indexes(sentence):
    indexes = ""
    counter=0
    for word in sentence.split(" "):
        indexes += str(counter) + "-" + str(counter + len(word)) + " "
        counter += len(word) + 1
    return indexes.rstrip()

def create_BIO(data):
    bio_labels = []
    for item in data:
        bio = ["O"] * len(item['request'].split())
        indexes = replace_words_with_indexes(item['request'])
        for entity in item['entities']:
            start = entity['start']
            end = entity['end']
            label_type = entity['category']
            for i, index in enumerate(indexes.split(" ")):
                if int(index.split("-")[0]) >= start and int(index.split("-")[1]) <= end:
                    if int(index.split("-")[0]) == start:
                        bio[i] = 'B-' + label_type 
                    else:
                        bio[i] = 'I-' + label_type
        bio_labels.append(' '.join(bio))
    return bio_labels

        

print (create_BIO(data))

['O O O O O B-DESTINATION I-DESTINATION O O B-DATE']


In [50]:
def BIO_converter(r, entities):
    to_replace = {} # needed to maintain all the NER to be replaced
    for i in entities:
        sub = r[i['start']+1:i['end']+2].split(' ') # 1 indexed values in entities
        if len(sub) > 1:
            vals = [f"B-{i['category']}"] + ([f"I-{i['category']}"] * (len(sub)-1))
        else:
            vals = [f"B-{i['category']}"]

        to_replace = to_replace | dict(zip(sub,vals))

    r = r.split(' ')
    r = [to_replace[i] if i in to_replace else 'O' for i in r ]
    return ' '.join(r)

js = {
        "request": "I want to fly to New York on the 13.3",
        "entities": [
          {"start": 16, "end": 23, "text": "New York", "category": "DESTINATION"},
          {"start": 32, "end": 35, "text": "13.3", "category": "DATE"}
        ]
      }
BIO_converter(js['request'], js['entities'])

'O O O O O B-DESTINATION I-DESTINATION O O B-DATE'

In [9]:
data

{'classes': ['DATE', 'ORIGIN', 'DESTINATION'],
 'annotations': [['אני רוצה לטוס לסינגפור\r',
   {'entities': [[14, 22, 'DESTINATION']]}],
  ['אני רוצה להזמין כרטיס לרומא\r', {'entities': [[22, 27, 'DESTINATION']]}],
  ["האם ניתן להזמין כרטיס טיסה דרך הצ'אטבוט?\r", {'entities': []}],
  ['ברצוני להזמין כרטיס טיסה מתל אביב לניו יורק לתאריך 2024-05-01.\r',
   {'entities': [[25, 33, 'ORIGIN'],
     [34, 43, 'DESTINATION'],
     [51, 61, 'DATE']]}],
  ['אני מעוניין לרכוש כרטיס טיסה שתצא מברצלונה ותגיע ליוון ב10.2\r',
   {'entities': [[34, 42, 'ORIGIN'],
     [49, 54, 'DESTINATION'],
     [55, 60, 'DATE']]}],
  ['בתאריך ה10 למרץ אני רוצה לטוס ליוטה\r',
   {'entities': [[7, 15, 'DATE'], [30, 35, 'DESTINATION']]}],
  ['באיזה מחיר ניתן להזמין כרטיס טיסה הלוך ושוב לפריז?\r',
   {'entities': [[44, 49, 'DESTINATION']]}],
  ['רציתי להזמין כרטיס טיסה לילד קטן.\r', {'entities': []}],
  ['ברצוני להזמין טיסה לספרד ליום חמישי\r',
   {'entities': [[19, 24, 'DESTINATION'], [30, 35, 'DATE']]}],
  ['כמה עולה

In [30]:
entity_name = ["ORIGIN", "DESTINATION", "DATE"]

train_data = data['annotations']
train_data = [tuple(i) for i in train_data]

train_data

[('אני רוצה לטוס לסינגפור\r', {'entities': [[14, 22, 'DESTINATION']]}),
 ('אני רוצה להזמין כרטיס לרומא\r', {'entities': [[22, 27, 'DESTINATION']]}),
 ("האם ניתן להזמין כרטיס טיסה דרך הצ'אטבוט?\r", {'entities': []}),
 ('ברצוני להזמין כרטיס טיסה מתל אביב לניו יורק לתאריך 2024-05-01.\r',
  {'entities': [[25, 33, 'ORIGIN'],
    [34, 43, 'DESTINATION'],
    [51, 61, 'DATE']]}),
 ('אני מעוניין לרכוש כרטיס טיסה שתצא מברצלונה ותגיע ליוון ב10.2\r',
  {'entities': [[34, 42, 'ORIGIN'],
    [49, 54, 'DESTINATION'],
    [55, 60, 'DATE']]}),
 ('בתאריך ה10 למרץ אני רוצה לטוס ליוטה\r',
  {'entities': [[7, 15, 'DATE'], [30, 35, 'DESTINATION']]}),
 ('באיזה מחיר ניתן להזמין כרטיס טיסה הלוך ושוב לפריז?\r',
  {'entities': [[44, 49, 'DESTINATION']]}),
 ('רציתי להזמין כרטיס טיסה לילד קטן.\r', {'entities': []}),
 ('ברצוני להזמין טיסה לספרד ליום חמישי\r',
  {'entities': [[19, 24, 'DESTINATION'], [30, 35, 'DATE']]}),
 ('כמה עולה כרטיס טיסה לברצלונה?\r', {'entities': [[20, 28, 'DESTINATION']]}),
 ('אני רוצה להזמ

In [35]:
counter = 0
for i in train_data:
    # print(i[1]['entities'])
    # print(len(i[1]['entities']))
    print(i[1]['entities'][2])
    # for entity in i[1]['entities']:
    #     print (entity[2])
    #     print("end")
    if len(i[1]['entities']) != 3:
        if len(i[1]['entities']) == 0:
            for entity in entity_name:
                i[1]['entities'] = (0, 0, entity)
        else:
            

    # if i[1]['entities'] == []:
    #     i[1]['entities'] = (0, 0, entity_name)
    # else:
        # i[1]['entities'][0] = tuple(i[1]['entities'][0])

IndexError: list index out of range